In [121]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [122]:
# %pip install catboost

In [123]:
matches = pd.read_csv('dataset/match_level_scorecard.csv')  # T20 games from past two years
batsmen = pd.read_csv('dataset/batsman_level_scorecard.csv')  # Batsmen scorecard details
bowlers = pd.read_csv('dataset/bowler_level_scorecard.csv')  # Bowler scorecard details
features= pd.read_csv('dataset/train_data.csv')  # Ready-to-use independent features

In [124]:
# calculating unique match id in matches
match_ids = matches['match id'].unique()
match_ids.shape

(1689,)

In [125]:
# shape of all the dataframes
matches.shape, batsmen.shape, bowlers.shape, features.shape

((1689, 30), (24483, 21), (18539, 18), (948, 23))

## Merging the datasets

In [126]:
batsmen_columns = batsmen.columns

In [127]:
batsmen_columns

Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
       'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
       'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
       'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
       'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
      dtype='object')

In [128]:
# changing the column names of batsmen dataframe like bowler_id

""" coumn names - Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
       'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
       'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
       'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
       'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
      dtype='object')
      
"""

batsmen.columns = ['match id', 'batsman', 'batsman_id', 'batsman_details','is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs','balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler','out_by_fielder', 'bowler_id_batsmen_scorecard', 'bowler_details_batsmen_scorecard', 'is_bowler_keeper_batsmen_scorecard','is_bowler_captain_batsmen_scorecard', 'strike_rate_batsmen_scorecard', 'Fours_batsmen_scorecard', 'Sixes_batsmen_scorecard', 'match_dt_batsmen_scorecard']

In [129]:
batsmen.head()

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id_batsmen_scorecard,bowler_details_batsmen_scorecard,is_bowler_keeper_batsmen_scorecard,is_bowler_captain_batsmen_scorecard,strike_rate_batsmen_scorecard,Fours_batsmen_scorecard,Sixes_batsmen_scorecard,match_dt_batsmen_scorecard
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,caught,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.00,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,caught,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.00,4.0,1.0,2021-01-01
2,8638034,HR Cr,4949790.0,NZ:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,9,10,12.1,caught,PF Yd,R Ra,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,90.00,NaN,NaN,2021-01-01
3,8638034,BR Hn,3834305.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,28,22,13.6,caught,HK Bt,FH An,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,127.27,3.0,NaN,2021-01-01
4,8638034,SC Kn,3776849.0,NZ:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,18,13,17.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.46,NaN,1.0,2021-01-01


In [130]:
bowlers.columns

Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'],
      dtype='object')

In [131]:


# Function to convert all values to strings and join them
def join_values(series):
    return ','.join(map(str, series))

# Aggregating the data
batsmen_agg = batsmen.groupby('match id').agg({
    'batsman': join_values,
    'batsman_id': join_values,
    'batsman_details': join_values,
    'is_batsman_captain': join_values,
    'is_batsman_keeper': join_values,
    'inning': join_values,
    'runs': join_values,
    'balls_faced': join_values,
    'over_faced_first': join_values,
    'wicket kind': join_values,
    'out_by_bowler': join_values,
    'out_by_fielder': join_values,
    'bowler_id_batsmen_scorecard': join_values,
    'bowler_details_batsmen_scorecard': join_values,
    'is_bowler_keeper_batsmen_scorecard': join_values,
    'is_bowler_captain_batsmen_scorecard': join_values,
    'strike_rate_batsmen_scorecard': join_values,
    'Fours_batsmen_scorecard': join_values,
    'Sixes_batsmen_scorecard': join_values,
    'match_dt_batsmen_scorecard': join_values
}).reset_index()


In [132]:
bowlers.columns

Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'],
      dtype='object')

In [133]:
# Function to convert all values to strings and join them
def join_values(series):
    return ','.join(map(str, series))

# Aggregating the data
bowlers_agg = bowlers.groupby('match id').agg({
    'bowler': join_values,
    'bowler_id': join_values,
    'bowler_details': join_values,
    'is_bowler_keeper': join_values,
    'is_bowler_captain': join_values,
    'inning': join_values,
    'runs': join_values,
    'wicket_count': join_values,
    'balls_bowled': join_values,
    'economy': join_values,
    'maiden': join_values,
    'dots': join_values,
    'Fours': join_values,
    'Sixes': join_values,
    'wides': join_values,
    'noballs': join_values,
    'match_dt': join_values
}).reset_index()

In [134]:
matches.shape

(1689, 30)

In [135]:
features.shape

(948, 23)

In [136]:
match_data.head()

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt_match,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt_batsmen,bowler,bowler_id_bowlers,bowler_details_bowlers,is_bowler_captain_bowlers,is_bowler_keeper_bowlers,inning_bowlers,runs_bowlers,wicket_count,balls_bowled,economy,maiden,dots,Fours_bowlers,Sixes_bowlers,wides,noballs,match_dt,team1_features,team1_id_features,team1_roster_ids_features,team2_features,team2_id_features,team2_roster_ids_features,winner_features,winner_id_features,toss winner_features,toss decision_features,venue_features,city_features,match_dt_features,lighting_features,series_name_features,season_features,ground_id_features,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,Hl Ph,Hl Ph,wickets,8.0,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,A Bi,Sh De,129,8,122,130.0,2.0,105.0,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,other_domestic,12634,NaN,HS Di,9373356.0,IND:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,6,5,1.1,run out,R Dn,SL Va,2035102.0,IND:Right-hand bat:Right-arm medium-fast:,0.0,1.0,120.0,1.0,NaN,2022-10-20,R Dn,2035102.0,IND:Right-hand bat:Right-arm medium-fast:,1.0,0.0,1,31,2,24,7.75,0,9.0,3,1,2,0,2022-10-20,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
1,9331181,Ba,Hl Ph,Hl Ph,wickets,8.0,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,A Bi,Sh De,129,8,122,130.0,2.0,105.0,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,other_domestic,12634,NaN,HS Di,9373356.0,IND:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,6,5,1.1,run out,R Dn,SL Va,2035102.0,IND:Right-hand bat:Right-arm medium-fast:,0.0,1.0,120.0,1.0,NaN,2022-10-20,VG Aa,8465057.0,IND:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,28,2,24,7.00,0,8.0,4,0,0,0,2022-10-20,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
2,9331181,Ba,Hl Ph,Hl Ph,wickets,8.0,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,A Bi,Sh De,129,8,122,130.0,2.0,105.0,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,other_domestic,12634,NaN,HS Di,9373356.0,IND:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,6,5,1.1,run out,R Dn,SL Va,2035102.0,IND:Right-hand bat:Right-arm medium-fast:,0.0,1.0,120.0,1.0,NaN,2022-10-20,KR Ke,7878989.0,IND:Right-hand bat:Right-arm offbreak:,0.0,0.0,2,13,1,17,4.59,0,5.0,0,0,1,0,2022-10-20,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
3,9331181,Ba,Hl Ph,Hl Ph,wickets,8.0,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,A Bi,Sh De,129,8,122,130.0,2.0,105.0,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035

In [137]:
match_data.to_csv('dataset/match_data.csv', index=False)

In [138]:
# combining the train_data with match_data , batsmen_agg and bowlers_agg data with only those match id which are present in train_data

train_data = pd.merge(features, matches, on='match id', how='inner')
train_data = pd.merge(train_data, batsmen_agg, on='match id', how='inner')
train_data = pd.merge(train_data, bowlers_agg, on='match id', how='inner')

In [139]:
train_data.shape

(948, 89)

In [140]:
# filling the missing values with 0
train_data.fillna(0, inplace=True)

In [141]:
# train_data.to_csv('dataset/updated_train_data.csv', index=False)

In [142]:
train_data.columns

Index(['match id', 'team1_x', 'team1_id_x', 'team1_roster_ids_x', 'team2_x',
       'team2_id_x', 'team2_roster_ids_x', 'winner_x', 'winner_id_x',
       'toss winner_x', 'toss decision_x', 'venue_x', 'city_x', 'match_dt_x',
       'lighting_x', 'series_name_x', 'season_x', 'ground_id_x',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_y', 'team2_y', 'winner_y', 'by',
       'win amount', 'toss winner_y', 'toss decision_y', 'venue_y', 'city_y',
       'match_dt_y', 'lighting_y', 'series_name_y', 'season_y', 'ground_id_y',
       'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets',
       'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'team1_id_y', 'team1_roster_ids_y', 'team2_id_y', 'team2_roster_ids_y',
       'series_type', 'winner_id_y', 'player_of_the_match_id', 'batsman',
       'batsman_id', 'batsman_details', 'is_batsman_captain',
       '

In [143]:
# show full columns in pandas
pd.set_option('display.max_columns', None)
train_data.head()

,match id,team1_x,team1_id_x,team1_roster_ids_x,team2_x,team2_id_x,team2_roster_ids_x,winner_x,winner_id_x,toss winner_x,toss decision_x,venue_x,city_x,match_dt_x,lighting_x,series_name_x,season_x,ground_id_x,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_y,team2_y,winner_y,by,win amount,toss winner_y,toss decision_y,venue_y,city_y,match_dt_y,lighting_y,series_name_y,season_y,ground_id_y,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id_y,team1_roster_ids_y,team2_id_y,team2_roster_ids_y,series_type,winner_id_y,player_of_the_match_id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning_x,runs_x,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id_batsmen_scorecard,bowler_details_batsmen_scorecard,is_bowler_keeper_batsmen_scorecard,is_bowler_captain_batsmen_scorecard,strike_rate_batsmen_scorecard,Fours_batsmen_scorecard,Sixes_batsmen_scorecard,match_dt_batsmen_scorecard,bowler,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,inning_y,runs_y,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,Ba,Hl Ph,Hl Ph,wickets,8.0,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,A Bi,Sh De,129,8,122,130.0,2.0,105.0,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,other_domestic,12634,0.0,"HS Di,NA Ra,KR Ke,VR An,KH Pa,AT Ru,P Ca,R Dn","9373356.0,7883504.0,7878989.0,2526390.0,329940...","IND:Left-hand bat:Right-arm medium-fast:,IND:L...","0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0","1,1,1,1,1,1,2,2","6,13,3,9,31,10,61,4","5,19,4,6,24,16,48,7","1.1,12.2,17.7,18.6,5.5,7.5,1.1,15.4","run out,caught,caught,nan,bowled,caught,nan,nan","R Dn,R Dn,Ah Vt,nan,Mk Dr,Mk Dr,nan,nan","SL Va,SL Va,Mk Dr,nan,nan,Dy Ri,nan,nan","2035102.0,2035102.0,nan,nan,nan,nan,nan,nan","IND:Right-hand bat:Right-arm medium-fast:,IND:...","0.0,0.0,nan,nan,nan,nan,nan,nan","1.0,1.0,nan,nan,nan,nan,nan,nan","120.0,68.42,75.0,150.0,129.17,62.5,127.08,57.14","1.0,1.0,nan,nan,5.0,1.0,4.0,nan","nan,nan,nan,1.0,nan,nan,2.0,nan","2022-10-20,2022-10-20,2022-10-20,2022-10-20,20...","R Dn,VG Aa,KR Ke,LI Ma,NA Ra,VR An","2035102.0,8465057.0,7878989.0,4017523.0,788350...","IND:Right-hand bat:Right-arm medium-fast:,IND:...","0.0,0.0,0.0,0.0,0.0,0.0","1.0,0.0,0.0,0.0,0.0,0.0","1,1,2,2,2,2","31,28,13,23,21,18","2,2,1,0,1,0","24,24,17,12,24,18","7.75,7.0,4.59,11.5,5.25,6.0","0,0,0,0,0,0","9.0,8.0,5.0,5.0,9.0,9.0","3,4,0,2,1,2","1,0,0,2,0,0","2,0,1,0,0,2","0,0,0,0,0,0","2022-10-20,2022-10-20,2022-10-20,2022-10-20,20..."
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,Ed,Wt Is,Ed,runs,1.0,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,JS Wn,N Dd,171,8,128,170.0,8.0,124.0,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,international,20,62432.0,"JJ Ry,T Bn,SW Bs,EG Mn,CJ Jn,LA Dn,AU Rd,S Md,...","2089079.0,6139370.0,2083409.0,172199.0,2022957...","ENG:Right-hand bat:None:,ENG:Right-hand bat:No...","0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1....","1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2","45,25,5,13,27,4,2,7,4,31,0,2,24,44,7,12,44,23,1,1","31,18,7,12,15,2,3,4,4,24,1,9,22,28,3,11,16,20,4,6","1.1,1.2,13.3,14.7,15.6,18.6,19.

In [144]:
# converting categorical columns to numerical columns
train_data=pd.get_dummies(train_data,columns=['team1_x','team1_roster_ids_x', 'team2_x',
       'team2_roster_ids_x', 
       'toss winner_x', 'toss decision_x', 'venue_x', 'city_x', 'match_dt_x',
       'lighting_x', 'series_name_x', 'season_x', 
        'team1_y', 'team2_y', 'winner_y', 'by',
        'toss winner_y', 'toss decision_y', 'venue_y', 'city_y',
       'match_dt_y', 'lighting_y', 'series_name_y', 'season_y',
       'umpire1', 'umpire2', 
       'team1_roster_ids_y', 'team2_roster_ids_y',
       'series_type',  'batsman',
        'batsman_details', 'is_batsman_captain',
       'is_batsman_keeper', 'inning_x', 'runs_x', 'balls_faced',
       'over_faced_first', 'wicket kind', 'out_by_bowler', 'out_by_fielder',
       'bowler_id_batsmen_scorecard', 'bowler_details_batsmen_scorecard',
       'is_bowler_keeper_batsmen_scorecard',
       'is_bowler_captain_batsmen_scorecard', 'strike_rate_batsmen_scorecard',
       'Fours_batsmen_scorecard', 'Sixes_batsmen_scorecard',
       'match_dt_batsmen_scorecard', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_keeper', 'is_bowler_captain', 'inning_y', 'runs_y',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'])

# train_data=pd.get_dummies(train_data,columns=['match id', 'team1_x', 'team1_id_x', 'team1_roster_ids_x', 'team2_x',
#        'team2_id_x', 'team2_roster_ids_x', 'winner_x', 'winner_id_x',
#        'toss winner_x', 'toss decision_x', 'venue_x', 'city_x', 'match_dt_x',
#        'lighting_x', 'series_name_x', 'season_x', 'ground_id_x',
#        'team_count_50runs_last15', 'team_winp_last5',
#        'team1only_avg_runs_last15', 'team1_winp_team2_last15',
#        'ground_avg_runs_last15', 'team1_y', 'team2_y', 'winner_y', 'by',
#        'win amount', 'toss winner_y', 'toss decision_y', 'venue_y', 'city_y',
#        'match_dt_y', 'lighting_y', 'series_name_y', 'season_y', 'ground_id_y',
#        'umpire1', 'umpire2', 'inning1_runs', 'inning1_wickets',
#        'inning1_balls', 'inning2_runs', 'inning2_wickets', 'inning2_balls',
#        'team1_id_y', 'team1_roster_ids_y', 'team2_id_y', 'team2_roster_ids_y',
#        'series_type', 'winner_id_y', 'player_of_the_match_id', 'batsman',
#        'batsman_id', 'batsman_details', 'is_batsman_captain',
#        'is_batsman_keeper', 'inning_x', 'runs_x', 'balls_faced',
#        'over_faced_first', 'wicket kind', 'out_by_bowler', 'out_by_fielder',
#        'bowler_id_batsmen_scorecard', 'bowler_details_batsmen_scorecard',
#        'is_bowler_keeper_batsmen_scorecard',
#        'is_bowler_captain_batsmen_scorecard', 'strike_rate_batsmen_scorecard',
#        'Fours_batsmen_scorecard', 'Sixes_batsmen_scorecard',
#        'match_dt_batsmen_scorecard', 'bowler', 'bowler_id', 'bowler_details',
#        'is_bowler_keeper', 'is_bowler_captain', 'inning_y', 'runs_y',
#        'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
#        'Sixes', 'wides', 'noballs', 'match_dt'])